In [13]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
torch.use_deterministic_algorithms(True)

In [14]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils import train, test, edgeindex2adj
from models import GCN, GAT, LP
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
pubmed = Planetoid(root='.', name='Pubmed')

In [15]:
# dataset = citeseer
# model = GCN(dataset.num_features, 24, dataset.num_classes)

dataset = cora
model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = pubmed
# model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# dataset = cora
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# dataset = pubmed
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [16]:
from models import ALP, GCN
dataset = cora
data = dataset[0]

# take k random training nodes for each class
k = 2
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

print(data.train_mask.sum())

tensor(14)


In [18]:
best_val_acc = final_test_acc = 0
for epoch in range(1, 200):
    loss = train(model, data, optimizer, loss='cross_entropy')
    train_acc, val_acc, tmp_test_acc = test(model, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 1.9669246673583984, Train: 0.5000, Val: 0.1140, Test: 0.1330
Epoch: 002, Loss: 1.9150588512420654, Train: 0.5000, Val: 0.1420, Test: 0.1510
Epoch: 003, Loss: 1.8717554807662964, Train: 0.5714, Val: 0.1640, Test: 0.1850
Epoch: 004, Loss: 1.8289309740066528, Train: 0.6429, Val: 0.1760, Test: 0.1960
Epoch: 005, Loss: 1.779698371887207, Train: 0.7143, Val: 0.1800, Test: 0.1920
Epoch: 006, Loss: 1.7275491952896118, Train: 0.7143, Val: 0.1780, Test: 0.1920
Epoch: 007, Loss: 1.6721351146697998, Train: 0.7857, Val: 0.1620, Test: 0.1920
Epoch: 008, Loss: 1.6143798828125, Train: 0.7857, Val: 0.1560, Test: 0.1920
Epoch: 009, Loss: 1.5547969341278076, Train: 0.7857, Val: 0.1520, Test: 0.1920
Epoch: 010, Loss: 1.4935821294784546, Train: 0.7857, Val: 0.1460, Test: 0.1920
Epoch: 011, Loss: 1.4316102266311646, Train: 0.7857, Val: 0.1440, Test: 0.1920
Epoch: 012, Loss: 1.3698276281356812, Train: 0.7857, Val: 0.1560, Test: 0.1920
Epoch: 013, Loss: 1.308699369430542, Train: 0.7857, Val:

In [7]:
# dataset = citeseer
# dataset = cora
dataset = pubmed

best_val_acc = final_test_acc = 0
best_l = best_a = 0

for l in [1, 2, 4, 8, 16, 32]:
    for a in [0.05, 0.1, 0.3, 0.6, 0.8, 0.9, 0.95, 0.99, 1]:
        model = LP(num_layers=l, alpha=a)
        outs = model.train(dataset)
        train_acc, val_acc, tmp_test_acc = model.test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
            best_l = l
            best_a = a
            
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}', f'Best l: {best_l}', f'Best a: {best_a}')

Best Val Acc: 0.7260 Test Acc: 0.7140 Best l: 16 Best a: 0.99


In [8]:
accs = model.test()
accs

[0.8833333333333333, 0.712, 0.707]

In [9]:
outs.shape

torch.Size([19717, 3])

In [10]:
outs.argmax(dim=-1)

tensor([1, 1, 0,  ..., 2, 0, 2])

In [11]:
dataset[0].y

tensor([1, 1, 0,  ..., 2, 0, 2])

In [12]:
dataset.num_classes

3